In [21]:
import collections

props = ["gndo:forename", "gndo:surname"]

cooc = collections.defaultdict(lambda : collections.defaultdict(collections.Counter))
tmp = collections.defaultdict(set)

def extract_literal(line):
    start = line.index('"')
    end = line.index('"', start + 1)
    return line[start + 1:end]

def bad_name(name):
    return " " in name or "." in name or "-" in name

def extract_from_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        count = 0
        for p in props:
            tmp[p] = set()
        for line in f:
            if "gndo:gndIdentifier" in line:
                for p in props:
                    if len(tmp[p]) > 1:
                        for name in tmp[p]:
                            cooc[p][name].update(tmp[p])
                    tmp[p] = set()
                # id = line.split(' ')[2].strip('"')
                count += 1
                if count % 10000 == 0:
                    print('.', end='')
                if count % 1000000 == 0:
                    print(' {}'.format(count))
            for p in props:
                if p in line:
                    name = extract_literal(line)
                    if not bad_name(name):
                        tmp[p].add(name)



In [22]:
extract_from_file('authorities-name_lds.ttl')

.................................................................................................... 1000000
.................................................................................................... 2000000
.................................................................................................... 3000000
.................................................................................................... 4000000
.................................................................................................... 5000000
.................................................................................................... 6000000
.................................................................................................... 7000000
....

In [25]:
extract_from_file('authorities-person_lds_20190213.ttl')

.................................................................................................... 1000000
.................................................................................................... 2000000
.................................................................................................... 3000000
.................................................................................................... 4000000
.................................................................................................... 5000000
.

In [26]:
len(cooc['gndo:forename'])

199591

In [29]:
cooc['gndo:forename']['Natalja'].most_common()
        

[('Natalja', 163),
 ('Natalia', 75),
 ('Natalija', 47),
 ('Natalʹja', 39),
 ('Natalya', 28),
 ("Natal'ja", 20),
 ('Nataliya', 6),
 ('Natalʹia', 5),
 ('Nataliia', 4),
 ('Nathalia', 3),
 ('Natalie', 3),
 ('Natal’ja', 2),
 ('Nathalie', 2),
 ('Natasha', 2),
 ('Natali︠a︡', 2),
 ('Natali͡a', 1),
 ('Natāriya', 1),
 ("Natal'Ja", 1),
 ('NATALJA', 1),
 ('Natalʹâ', 1),
 ('Natalʹi︠a︡', 1),
 ('Natalʹlja', 1),
 ('Natallja', 1),
 ('Natal´ia', 1),
 ('Natalʹya', 1),
 ("Natal'ia", 1),
 ('Natali', 1),
 ('Natascha', 1),
 ('Natala', 1),
 ('Kamaliyha', 1),
 ('Kamalija', 1),
 ('Natalka', 1),
 ('Nadja', 1),
 ('Natalii͡a', 1),
 ('Natali︠a', 1)]